# Maintainance

This package is intended to be simple for users to use, but also for maintainers to maintain (with some kind of maintenance specifically prioritised).

There are two ways we might want to improve this package:

1. Adding more 'outputs' (e.g. more types of results from the simulation inputs, using the underlying tax-benefit models).
2. Everything else related to the underlying way that the package uses the models.

This package is meant to make (1) as easy as possible, which may come at the expense of some ease of maintaining (2).

## Model outputs

The package is centered around an 'output tree': a hierarchical nested dictionary of outputs from the model, of the type that the user asked for. For example, if we are doing a macro analysis, we might expect something like:

```yaml
macro:
    baseline:
        tax_revenue: 100 billion
    reform:
        tax_revenue: 200 billion
    comparison:
        tax_revenue_impact: 100 billion
```

If you look at the file structure of the repo, you'll see a folder `policyengine/outputs/`. This is where all the behaviour to deal with the models is housed. It's designed so that maintainers should be able to add something new, e.g. `macro/comparison/deep_poverty_impact` or `macro/single/deep_poverty_rate`, without needing to know how the rest of the repo works. Each file here is dynamically loaded into the tree (notice how `macro/comparison/tax_revenue_impact.py` aligns with that example- not a coincidence!), and must contain a single function, with the same name as the filename, that takes `policyengine.Simulation` as an argument. `policyengine.Simulation` has all the tax-benefit models initialised that the function might need, as well as all the metadata the user has passing in about their modelling request.

There's a slight caveat here that not everything aligns exactly with this folder structure. At the top level, we have household and macro, but one layer down, we have 'single' (e.g. 'poverty rate') and 'comparison' (e.g. 'change in poverty rate'). At runtime, 'single' is duplicated into 'baseline' and 'reform' with the operations duplicated. For example, here's the output tree under that example from the introduction.

In [1]:
from policyengine import Simulation

sim = Simulation(
    country="uk",
    scope="macro",
    data="enhanced_frs",
    time_period=2025,
    reform={
        "gov.hmrc.income_tax.allowances.personal_allowance.amount": {
            "2025": 0,
        }
    }
)
sim.calculate("macro")

{'comparison': {'inequality': {'gini': {'baseline': 0.3461090503003493,
    'reform': 0.35650449384824956},
   'top_10_pct_share': {'baseline': 0.30135235177122566,
    'reform': 0.3156585625028572},
   'top_1_pct_share': {'baseline': 0.12847120313264457,
    'reform': 0.1376559353368581}},
  'labor_supply': {'earnings': {'baseline': 1353268750344.8718,
    'reform': 1353268750344.8718,
    'change': 0.0,
    'rel_change': 0.0},
   'fte': {'baseline': 32671795.115284607,
    'reform': 32671795.115284607,
    'change': 0.0,
    'rel_change': 0.0},
   'substitution_lsr': 0,
   'income_lsr': 0,
   'relative_lsr': {'income': 0.0, 'substitution': 0.0},
   'decile': {'average': {'income': {-1: 0.0,
      1: 0.0,
      2: 0.0,
      3: 0.0,
      4: 0.0,
      5: 0.0,
      6: 0.0,
      7: 0.0,
      8: 0.0,
      9: 0.0,
      10: 0.0},
     'substitution': {-1: 0.0,
      1: 0.0,
      2: 0.0,
      3: 0.0,
      4: 0.0,
      5: 0.0,
      6: 0.0,
      7: 0.0,
      8: 0.0,
      9: 0.0,